<a href="https://colab.research.google.com/github/DhekerKacem/FuturVisionTransport/blob/main/modelisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install imgaug

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

### Chargement des bibliothèques

In [ ]:
import os
import numpy as np
import cv2
from tensorflow.keras.utils import Sequence, to_categorical
import imgaug as ia
from imgaug import augmenters as iaa
from imgaug.augmentables.segmaps import SegmentationMapsOnImage
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Dropout, Conv2DTranspose, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Dropout
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras import backend as K

### Définition de la métrique personnalisée

In [ ]:
class CustomMeanIoU(tf.keras.metrics.Metric):
    def __init__(self, name='mean_iou', num_classes=8, **kwargs):
        super(CustomMeanIoU, self).__init__(name=name, **kwargs)
        self.num_classes = num_classes
        self.total_iou = self.add_weight(name='total_iou', initializer='zeros')
        self.count_classes = self.add_weight(name='count_classes', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        if len(y_true.shape) == 4:
            y_true = tf.argmax(y_true, axis=-1)
            y_pred = tf.argmax(y_pred, axis=-1)

        mean_iou = 0.0
        class_count = 0.0

        for i in range(self.num_classes):
            true_class = K.cast(K.equal(y_true, i), dtype=tf.float32)
            pred_class = K.cast(K.equal(y_pred, i), dtype=tf.float32)

            intersection = K.sum(true_class * pred_class)
            union = K.sum(true_class) + K.sum(pred_class) - intersection

            iou = intersection / (union + K.epsilon())
            condition = K.equal(union, 0)
            mean_iou = K.switch(condition, mean_iou, mean_iou + iou)
            class_count = K.switch(condition, class_count, class_count + 1)

        self.total_iou.assign_add(mean_iou)
        self.count_classes.assign_add(class_count)

    def result(self):
        return self.total_iou / (self.count_classes + K.epsilon())

    def reset_state(self):
        self.total_iou.assign(0.0)
        self.count_classes.assign(0.0)

### Modèle Mini_Unet avec augmentation

#### Générateur de données

In [ ]:
class DataLoadGenerator(Sequence):
    CATS = {
        'void': [0, 1, 2, 3, 4, 5, 6],
        'flat': [7, 8, 9, 10],
        'construction': [11, 12, 13, 14, 15, 16],
        'object': [17, 18, 19, 20],
        'nature': [21, 22],
        'sky': [23],
        'human': [24, 25],
        'vehicle': [26, 27, 28, 29, 30, 31, 32, 33, -1]
    }

    def __init__(self, image_paths, mask_paths, crop_x, crop_y, batch_size, augment=False, shuffle=True):
        """
        Générateur de données avec ou sans augmentation des images
        """
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.crop_x = crop_x
        self.crop_y = crop_y
        self.batch_size = batch_size
        self.augment = augment
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.image_paths) / self.batch_size))

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.image_paths))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __getitem__(self, idx):
        batch_x = self.image_paths[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.mask_paths[idx * self.batch_size:(idx + 1) * self.batch_size]
        x,y= self._prepare_data(batch_x,batch_y)
        return x,y

    def _prepare_data(self, image_paths, mask_paths):
        x = []
        y = []
        for idx, (img_path, mask_path) in enumerate(zip(image_paths, mask_paths)):
            img, mask = self._get_image_and_mask(img_path, mask_path)
            if self.augment:
                img, mask = self._augment_image(img, mask)
            x.append(img)
            y.append(mask)
        x = np.array(x, dtype=np.float32) / 255.0
        y = np.array(y, dtype=np.uint8)
        y = np.expand_dims(y, axis=-1)
        y = to_categorical(y, num_classes=8)
        return x, y

    def _get_image_and_mask(self, img_path, mask_path):
        img = cv2.imread(img_path)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (self.crop_x, self.crop_y))
        mask = cv2.resize(mask, (self.crop_x, self.crop_y), interpolation=cv2.INTER_NEAREST)
        mask = self._convert_mask(mask)
        return img, mask

    def _convert_mask(self, img):
        mask = np.zeros((img.shape[0], img.shape[1], 8), dtype=np.uint8)
        for i in range(-1, 34):
            if i in self.CATS['void']:
                mask[:, :, 0] = np.logical_or(mask[:, :, 0], (img == i))
            elif i in self.CATS['flat']:
                mask[:, :, 1] = np.logical_or(mask[:, :, 1], (img == i))
            elif i in self.CATS['construction']:
                mask[:, :, 2] = np.logical_or(mask[:, :, 2], (img == i))
            elif i in self.CATS['object']:
                mask[:, :, 3] = np.logical_or(mask[:, :, 3], (img == i))
            elif i in self.CATS['nature']:
                mask[:, :, 4] = np.logical_or(mask[:, :, 4], (img == i))
            elif i in self.CATS['sky']:
                mask[:, :, 5] = np.logical_or(mask[:, :, 5], (img == i))
            elif i in self.CATS['human']:
                mask[:, :, 6] = np.logical_or(mask[:, :, 6], (img == i))
            elif i in self.CATS['vehicle']:
                mask[:, :, 7] = np.logical_or(mask[:, :, 7], (img == i))
        return np.argmax(mask, axis=2).astype('uint8')

    def _augment_image(self, img, mask):
        segmap = SegmentationMapsOnImage(mask, shape=img.shape)

        seq = iaa.Sequential([
            iaa.Affine(rotate=(-10, 10)),
            iaa.GammaContrast((0.5, 2.0)),
            iaa.Fliplr(0.5),
            iaa.CropAndPad(px=(-10, 10)),
            iaa.AdditiveGaussianNoise(scale=(0, 0.05*255))
        ])

        ia.seed(2)
        img_aug, segmap_aug = seq(image=img, segmentation_maps=segmap)
        final_img = img_aug
        final_mask = segmap_aug.get_arr()

        return final_img, final_mask

# Fonction pour obtenir les chemins des images et des masques
def get_image_and_mask_paths(image_base_dir, mask_base_dir):
    cities = os.listdir(image_base_dir)
    image_paths = []
    mask_paths = []
    for city in cities:
        city_image_dir = os.path.join(image_base_dir, city)
        city_mask_dir = os.path.join(mask_base_dir, city)
        images = os.listdir(city_image_dir)
        for img in images:
            mask = img.replace('_leftImg8bit.png', '_gtFine_color.png')
            mask_path = os.path.join(city_mask_dir, mask)
            if os.path.exists(mask_path):
                image_paths.append(os.path.join(city_image_dir, img))
                mask_paths.append(mask_path)
            else:
                print(f"Mask not found for image: {img}")
    print(f"Total images loaded from {image_base_dir}: {len(image_paths)}")
    print(f"Total masks loaded from {mask_base_dir}: {len(mask_paths)}")
    return image_paths, mask_paths

# Obtenir les chemins pour l'entraînement
train_image_base_dir = '/content/drive/My Drive//data/leftImg8bit/train'
train_mask_base_dir = '/content/drive/My Drive//data/gtFine/train'
train_image_paths, train_mask_paths = get_image_and_mask_paths(train_image_base_dir, train_mask_base_dir)

# Obtenir les chemins pour la validation
val_image_base_dir = '/content/drive/My Drive//data/leftImg8bit/val'
val_mask_base_dir = '/content/drive/My Drive//data/gtFine/val'
val_image_paths, val_mask_paths = get_image_and_mask_paths(val_image_base_dir, val_mask_base_dir)

# Obtenir les chemins pour le test
test_image_base_dir = '/content/drive/My Drive/data/leftImg8bit/test'
test_mask_base_dir = '/content/drive/My Drive//data/gtFine/test'
test_image_paths, test_mask_paths = get_image_and_mask_paths(test_image_base_dir, test_mask_base_dir)

# Création des DataLoaders
train_loader = DataLoadGenerator(train_image_paths, train_mask_paths, crop_x=512, crop_y=512, batch_size=8, augment=True)
val_loader = DataLoadGenerator(val_image_paths, val_mask_paths, crop_x=512, crop_y=512, batch_size=8, augment=False, shuffle=False)
test_loader = DataLoadGenerator(test_image_paths, test_mask_paths, crop_x=512, crop_y=512, batch_size=8, augment=False, shuffle=False)

print(f"Total batches in train_loader: {len(train_loader)}")
print(f"Total batches in val_loader: {len(val_loader)}")
print(f"Total batches in test_loader: {len(test_loader)}")

# Pour tester un lot de données d'entraînement et afficher la forme du masque d'origine :
x_train, y_train = train_loader[0]
print(x_train.shape, y_train.shape)  # Affiche la forme des lots de données générés

# Pour tester un lot de données de validation :
x_val, y_val = val_loader[0]
print(x_val.shape, y_val.shape)  # Affiche la forme des lots de données générés

# Pour tester un lot de données de test :
x_test, y_test = test_loader[0]
print(x_test.shape, y_test.shape)  # Affiche la forme des lots de données générés

Total images loaded from /content/drive/My Drive//data/leftImg8bit/train: 2975
Total masks loaded from /content/drive/My Drive//data/gtFine/train: 2975
Total images loaded from /content/drive/My Drive//data/leftImg8bit/val: 500
Total masks loaded from /content/drive/My Drive//data/gtFine/val: 500
Total images loaded from /content/drive/My Drive/data/leftImg8bit/test: 1525
Total masks loaded from /content/drive/My Drive//data/gtFine/test: 1525
Total batches in train_loader: 372
Total batches in val_loader: 63
Total batches in test_loader: 191
(8, 512, 512, 3) (8, 512, 512, 8)
(8, 512, 512, 3) (8, 512, 512, 8)
(8, 512, 512, 3) (8, 512, 512, 8)


#### Modèle Mini_Unet

In [ ]:
def mini_unet(input_size=(512, 512, 3), n_classes=8):
    inputs = Input(input_size)
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv4)

    up1 = concatenate([UpSampling2D(size=(2, 2))(conv4), conv3], axis=-1)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(up1)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

    up2 = concatenate([UpSampling2D(size=(2, 2))(conv5), conv2], axis=-1)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up2)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

    up3 = concatenate([UpSampling2D(size=(2, 2))(conv6), conv1], axis=-1)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up3)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

    output = Conv2D(n_classes, (1, 1), activation='softmax')(conv7)

    model = Model(inputs=inputs, outputs=output)
    return model

# Créer le modèle avec input_size = (512, 512, 3)
model = mini_unet(input_size=(512, 512, 3), n_classes=8)
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=[CustomMeanIoU(num_classes=8)])


# Définir les callbacks
model_checkpoint = ModelCheckpoint('mini_unet_SA.weights.h5', monitor='val_loss', save_best_only=True, save_weights_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Définir steps_per_epoch et validation_steps
# Remplacer train_loader et val_loader par vos générateurs de données ou DataLoader
steps_per_epoch = len(train_loader)
validation_steps = len(val_loader)

# Assurez-vous que steps_per_epoch et validation_steps sont des entiers positifs
if steps_per_epoch <= 0 or validation_steps <= 0:
    raise ValueError("steps_per_epoch et validation_steps doivent être des entiers positifs")

In [ ]:
# Vérifier que les formes des données correspondent à celles attendues par le modèle
print("Forme des données d'entraînement :", x_train.shape, y_train.shape)
print("Forme des données de validation :", x_val.shape, y_val.shape)


# Calculer le nombre de lots de données pour l'entraînement et la validation
steps_per_epoch = len(train_loader)
validation_steps = len(val_loader)

# Démarrer l'entraînement du modèle
history = model.fit(
    train_loader,
    workers=30,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_loader,
    validation_steps=validation_steps,
    epochs=50,
    callbacks=[model_checkpoint, early_stopping]
)

Forme des données d'entraînement : (8, 512, 512, 3) (8, 512, 512, 8)
Forme des données de validation : (8, 512, 512, 3) (8, 512, 512, 8)
Epoch 1/50
372/372 [==============================] - 144s 369ms/step - loss: 0.4814 - mean_iou: 0.2585 - val_loss: 0.3335 - val_mean_iou: 0.2440
Epoch 2/50
372/372 [==============================] - 139s 365ms/step - loss: 0.3187 - mean_iou: 0.2601 - val_loss: 0.3259 - val_mean_iou: 0.2440
Epoch 3/50
372/372 [==============================] - 139s 366ms/step - loss: 0.3111 - mean_iou: 0.2601 - val_loss: 0.3112 - val_mean_iou: 0.2440
Epoch 4/50
372/372 [==============================] - 139s 364ms/step - loss: 0.2937 - mean_iou: 0.2601 - val_loss: 0.3199 - val_mean_iou: 0.2440
Epoch 5/50
372/372 [==============================] - 138s 364ms/step - loss: 0.2806 - mean_iou: 0.2608 - val_loss: 0.3132 - val_mean_iou: 0.2440
Epoch 6/50
372/372 [==============================] - 139s 365ms/step - loss: 0.2704 - mean_iou: 0.2623 - val_loss: 0.2878 - val_mean

### Modèle sans augmentation

#### Générateur de données

In [ ]:
class DataLoadGeneratorSA(Sequence):
    CATS = {
        'void': [0, 1, 2, 3, 4, 5, 6],
        'flat': [7, 8, 9, 10],
        'construction': [11, 12, 13, 14, 15, 16],
        'object': [17, 18, 19, 20],
        'nature': [21, 22],
        'sky': [23],
        'human': [24, 25],
        'vehicle': [26, 27, 28, 29, 30, 31, 32, 33, -1]
    }

    def __init__(self, image_paths, mask_paths, crop_x, crop_y, batch_size, shuffle=True):
        """
        Générateur de données sans augmentation des images
        """
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.crop_x = crop_x
        self.crop_y = crop_y
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.image_paths) / self.batch_size))

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.image_paths))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __getitem__(self, idx):
        batch_x = self.image_paths[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.mask_paths[idx * self.batch_size:(idx + 1) * self.batch_size]
        x, y = self._prepare_data(batch_x, batch_y)
        return x, y

    def _prepare_data(self, image_paths, mask_paths):
        x = []
        y = []
        for img_path, mask_path in zip(image_paths, mask_paths):
            img, mask = self._get_image_and_mask(img_path, mask_path)
            x.append(img)
            y.append(mask)
        x = np.array(x, dtype=np.float32) / 255.0
        y = np.array(y, dtype=np.uint8)
        y = np.expand_dims(y, axis=-1)
        y = to_categorical(y, num_classes=8)
        return x, y

    def _get_image_and_mask(self, img_path, mask_path):
        img = cv2.imread(img_path)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (self.crop_x, self.crop_y))
        mask = cv2.resize(mask, (self.crop_x, self.crop_y), interpolation=cv2.INTER_NEAREST)
        mask = self._convert_mask(mask)
        return img, mask

    def _convert_mask(self, img):
        mask = np.zeros((img.shape[0], img.shape[1], 8), dtype=np.uint8)
        for i in range(-1, 34):
            if i in self.CATS['void']:
                mask[:, :, 0] = np.logical_or(mask[:, :, 0], (img == i))
            elif i in self.CATS['flat']:
                mask[:, :, 1] = np.logical_or(mask[:, :, 1], (img == i))
            elif i in self.CATS['construction']:
                mask[:, :, 2] = np.logical_or(mask[:, :, 2], (img == i))
            elif i in self.CATS['object']:
                mask[:, :, 3] = np.logical_or(mask[:, :, 3], (img == i))
            elif i in self.CATS['nature']:
                mask[:, :, 4] = np.logical_or(mask[:, :, 4], (img == i))
            elif i in self.CATS['sky']:
                mask[:, :, 5] = np.logical_or(mask[:, :, 5], (img == i))
            elif i in self.CATS['human']:
                mask[:, :, 6] = np.logical_or(mask[:, :, 6], (img == i))
            elif i in self.CATS['vehicle']:
                mask[:, :, 7] = np.logical_or(mask[:, :, 7], (img == i))
        return np.argmax(mask, axis=2).astype('uint8')

# Fonction pour obtenir les chemins des images et des masques
def get_image_and_mask_paths(image_base_dir, mask_base_dir):
    cities = os.listdir(image_base_dir)
    image_paths = []
    mask_paths = []
    for city in cities:
        city_image_dir = os.path.join(image_base_dir, city)
        city_mask_dir = os.path.join(mask_base_dir, city)
        images = os.listdir(city_image_dir)
        for img in images:
            mask = img.replace('_leftImg8bit.png', '_gtFine_color.png')
            mask_path = os.path.join(city_mask_dir, mask)
            if os.path.exists(mask_path):
                image_paths.append(os.path.join(city_image_dir, img))
                mask_paths.append(mask_path)
            else:
                print(f"Mask not found for image: {img}")
    print(f"Total images loaded from {image_base_dir}: {len(image_paths)}")
    print(f"Total masks loaded from {mask_base_dir}: {len(mask_paths)}")
    return image_paths, mask_paths

# Obtenir les chemins pour l'entraînement
train_image_base_dir = '/content/drive/My Drive//data/leftImg8bit/train'
train_mask_base_dir = '/content/drive/My Drive//data/gtFine/train'
train_image_paths, train_mask_paths = get_image_and_mask_paths(train_image_base_dir, train_mask_base_dir)

# Obtenir les chemins pour la validation
val_image_base_dir = '/content/drive/My Drive//data/leftImg8bit/val'
val_mask_base_dir = '/content/drive/My Drive//data/gtFine/val'
val_image_paths, val_mask_paths = get_image_and_mask_paths(val_image_base_dir, val_mask_base_dir)

# Obtenir les chemins pour le test
test_image_base_dir = '/content/drive/My Drive/data/leftImg8bit/test'
test_mask_base_dir = '/content/drive/My Drive//data/gtFine/test'
test_image_paths, test_mask_paths = get_image_and_mask_paths(test_image_base_dir, test_mask_base_dir)

# Création des DataLoaders sans augmentation
train_loaderSA = DataLoadGeneratorSA(train_image_paths, train_mask_paths, crop_x=512, crop_y=512, batch_size=8, shuffle=True)
val_loaderSA = DataLoadGeneratorSA(val_image_paths, val_mask_paths, crop_x=512, crop_y=512, batch_size=8, shuffle=False)
test_loaderSA = DataLoadGeneratorSA(test_image_paths, test_mask_paths, crop_x=512, crop_y=512, batch_size=8, shuffle=False)

print(f"Total batches in train_loader: {len(train_loaderSA)}")
print(f"Total batches in val_loader: {len(val_loaderSA)}")
print(f"Total batches in test_loader: {len(test_loaderSA)}")

# Pour tester un lot de données d'entraînement et afficher la forme du masque d'origine :
x_train_SA, y_train_SA = train_loaderSA[0]
print(x_train.shape, y_train.shape)  # Affiche la forme des lots de données générés

# Pour tester un lot de données de validation :
x_val_SA, y_val_SA = val_loaderSA[0]
print(x_val.shape, y_val.shape)  # Affiche la forme des lots de données générés

# Pour tester un lot de données de test :
x_test_SA, y_test_SA = test_loaderSA[0]
print(x_test.shape, y_test.shape)  # Affiche la forme des lots de données générés


Total images loaded from /content/drive/My Drive//data/leftImg8bit/train: 2975
Total masks loaded from /content/drive/My Drive//data/gtFine/train: 2975
Total images loaded from /content/drive/My Drive//data/leftImg8bit/val: 500
Total masks loaded from /content/drive/My Drive//data/gtFine/val: 500
Total images loaded from /content/drive/My Drive/data/leftImg8bit/test: 1525
Total masks loaded from /content/drive/My Drive//data/gtFine/test: 1525
Total batches in train_loader: 372
Total batches in val_loader: 63
Total batches in test_loader: 191
(8, 512, 512, 3) (8, 512, 512, 8)
(8, 512, 512, 3) (8, 512, 512, 8)
(8, 512, 512, 3) (8, 512, 512, 8)


#### Modèle Mini_Unet

In [ ]:
def mini_unet_SA(input_size=(512, 512, 3), n_classes=8):
    inputs = Input(input_size)
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv4)

    up1 = concatenate([UpSampling2D(size=(2, 2))(conv4), conv3], axis=-1)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(up1)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

    up2 = concatenate([UpSampling2D(size=(2, 2))(conv5), conv2], axis=-1)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up2)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

    up3 = concatenate([UpSampling2D(size=(2, 2))(conv6), conv1], axis=-1)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up3)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

    output = Conv2D(n_classes, (1, 1), activation='softmax')(conv7)

    model = Model(inputs=inputs, outputs=output)
    return model

# Créer le modèle avec input_size = (512, 512, 3)
model_SA = mini_unet_SA(input_size=(512, 512, 3), n_classes=8)
model_SA.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=[CustomMeanIoU(num_classes=8)])


# Définir les callbacks
model_checkpoint_SA = ModelCheckpoint('mini_unet_SA.weights.h5', monitor='val_loss', save_best_only=True, save_weights_only=True)
early_stopping_SA = EarlyStopping(monitor='val_loss', patience=10)

# Définir steps_per_epoch et validation_steps
# Remplacer train_loader et val_loader par vos générateurs de données ou DataLoader
steps_per_epoch = len(train_loader_SA)
validation_steps = len(val_loader_SA)

# Assurez-vous que steps_per_epoch et validation_steps sont des entiers positifs
if steps_per_epoch <= 0 or validation_steps <= 0:
    raise ValueError("steps_per_epoch et validation_steps doivent être des entiers positifs")

In [ ]:
model.summary()

#### Entrainement du modèle

In [ ]:
# Vérifier que les formes des données correspondent à celles attendues par le modèle
print("Forme des données d'entraînement :", x_train_SA.shape, y_train_SA.shape)
print("Forme des données de validation :", x_val_SA.shape, y_val_SA.shape)
print("Forme des données de test :", x_test_SA.shape, y_test_SA.shape)


# Calculer le nombre de lots de données pour l'entraînement et la validation
steps_per_epoch = len(train_loaderSA)
validation_steps = len(val_loaderSA)

# Démarrer l'entraînement du modèle
history = model_SA.fit(
    train_loaderSA,
    workers=30,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_loaderSA,
    validation_steps=validation_steps,
    epochs=50,
    callbacks=[model_checkpoint_SA, early_stopping_SA]
)

### Modèle VGG16 sans augmentation


In [ ]:
def vgg16_sa(input_size=(512, 512, 3), n_classes=8):
    inputs = Input(input_size)

    # Redimensionner les images d'entrée à (224, 224, 3) pour VGG16
    resized_inputs = Lambda(lambda x: tf.image.resize(x, (224, 224)))(inputs)
    preprocessed_inputs = preprocess_input(resized_inputs)

    # Encoder (VGG16)
    vgg16 = VGG16(include_top=False, weights='imagenet', input_tensor=inputs)
    vgg16.trainable = False  # Freeze the VGG16 layers

    block4_pool = vgg16.get_layer("block4_pool").output
    block5_conv3 = vgg16.get_layer("block5_conv3").output

    # Decoder
    up1 = Conv2DTranspose(512, (3, 3), strides=(2, 2), padding='same')(block5_conv3)
    up1 = Conv2D(512, (3, 3), activation='relu', padding='same')(up1)

    # Redimensionnement du block4_pool pour correspondre à up1
    block4_pool_resized =  Lambda(lambda x: tf.image.resize(x, (up1.shape[1], up1.shape[2])))(block4_pool)
    up1 = concatenate([up1, block4_pool_resized], axis=-1)

    up2 = Conv2DTranspose(256, (3, 3), strides=(2, 2), padding='same')(up1)
    up2 = Conv2D(256, (3, 3), activation='relu', padding='same')(up2)

    up3 = Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same')(up2)
    up3 = Conv2D(128, (3, 3), activation='relu', padding='same')(up3)

    up4 = Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same')(up3)
    up4 = Conv2D(64, (3, 3), activation='relu', padding='same')(up4)

    up5 = Conv2DTranspose(32, (3, 3), strides=(2, 2), padding='same')(up4)
    up5 = Conv2D(32, (3, 3), activation='relu', padding='same')(up5)

    # Redimensionner pour revenir à la taille d'origine (512, 512)
    final_upsampling = Lambda(lambda x: tf.image.resize(x, (512, 512)))(up5)

    output = Conv2D(n_classes, (1, 1), activation='softmax')(final_upsampling)

    model = Model(inputs=inputs, outputs=output)
    return model

# Créer le modèle avec input_size = (512, 512, 3)
model_vgg_sa = vgg16_sa(input_size=(512, 512, 3), n_classes=8)
model_vgg_sa.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=[CustomMeanIoU(num_classes=8)])

# Définir les callbacks pour surveiller dice_coefficient
model_checkpoint_vgg_sa = ModelCheckpoint('vgg16_unet.weights.h5', monitor='val_loss', save_best_only=True, save_weights_only=True)
early_stopping_vgg_sa = EarlyStopping(monitor='val_loss', patience=10, mode='min')

In [ ]:
import gc
gc.collect()

116

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
# Vérifier que les formes des données correspondent à celles attendues par le modèle
print("Forme des données d'entraînement :", x_train_SA.shape, y_train_SA.shape)
print("Forme des données de validation :", x_val_SA.shape, y_val_SA.shape)
print("Forme des données de test :", x_test_SA.shape, y_test_SA.shape)


# Calculer le nombre de lots de données pour l'entraînement et la validation
steps_per_epoch = len(train_loaderSA)
validation_steps = len(val_loaderSA)

# Démarrer l'entraînement du modèle
history = model_vgg_sa.fit(
    train_loaderSA,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_loaderSA,
    validation_steps=validation_steps,
    epochs=20,
    callbacks=[model_checkpoint_vgg_sa, early_stopping_vgg_sa]
)

Forme des données d'entraînement : (8, 512, 512, 3) (8, 512, 512, 8)
Forme des données de validation : (8, 512, 512, 3) (8, 512, 512, 8)
Forme des données de test : (8, 512, 512, 3) (8, 512, 512, 8)
Epoch 1/20
  1/372 [..............................] - ETA: 17:08:03 - loss: 2.0760 - mean_iou: 0.0136

### Modèle VGG16 avec augmentation

In [ ]:
def vgg16(input_size=(512, 512, 3), n_classes=8):
    inputs = Input(input_size)

    # Redimensionner les images d'entrée à (224, 224, 3) pour VGG16
    resized_inputs = Lambda(lambda x: tf.image.resize(x, (224, 224)))(inputs)
    preprocessed_inputs = preprocess_input(resized_inputs)

    # Encoder (VGG16)
    vgg16 = VGG16(include_top=False, weights='imagenet', input_tensor=inputs)
    vgg16.trainable = False  # Freeze the VGG16 layers

    block4_pool = vgg16.get_layer("block4_pool").output
    block5_conv3 = vgg16.get_layer("block5_conv3").output

    # Decoder
    up1 = Conv2DTranspose(512, (3, 3), strides=(2, 2), padding='same')(block5_conv3)
    up1 = Conv2D(512, (3, 3), activation='relu', padding='same')(up1)

    # Redimensionnement du block4_pool pour correspondre à up1
    block4_pool_resized =  Lambda(lambda x: tf.image.resize(x, (up1.shape[1], up1.shape[2])))(block4_pool)
    up1 = concatenate([up1, block4_pool_resized], axis=-1)

    up2 = Conv2DTranspose(256, (3, 3), strides=(2, 2), padding='same')(up1)
    up2 = Conv2D(256, (3, 3), activation='relu', padding='same')(up2)

    up3 = Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same')(up2)
    up3 = Conv2D(128, (3, 3), activation='relu', padding='same')(up3)

    up4 = Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same')(up3)
    up4 = Conv2D(64, (3, 3), activation='relu', padding='same')(up4)

    up5 = Conv2DTranspose(32, (3, 3), strides=(2, 2), padding='same')(up4)
    up5 = Conv2D(32, (3, 3), activation='relu', padding='same')(up5)

    # Redimensionner pour revenir à la taille d'origine (512, 512)
    final_upsampling = Lambda(lambda x: tf.image.resize(x, (512, 512)))(up5)

    output = Conv2D(n_classes, (1, 1), activation='softmax')(final_upsampling)

    model = Model(inputs=inputs, outputs=output)
    return model

# Créer le modèle avec input_size = (512, 512, 3)
model_vgg = vgg16(input_size=(512, 512, 3), n_classes=8)
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=[CustomMeanIoU(num_classes=8)])

# Définir les callbacks pour surveiller dice_coefficient
model_checkpoint_vgg = ModelCheckpoint('vgg16_unet.weights.h5', monitor='val_loss', save_best_only=True, save_weights_only=True)
early_stopping_vgg = EarlyStopping(monitor='val_loss', patience=10, mode='min')

In [ ]:
# Vérifier que les formes des données correspondent à celles attendues par le modèle
print("Forme des données d'entraînement :", x_train.shape, y_train.shape)
print("Forme des données de validation :", x_val.shape, y_val.shape)
print("Forme des données de test :", x_test.shape, y_test.shape)


# Calculer le nombre de lots de données pour l'entraînement et la validation
steps_per_epoch = len(train_loader)
validation_steps = len(val_loader)

# Démarrer l'entraînement du modèle
history = model_vgg.fit(
    train_loader,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_loader,
    validation_steps=validation_steps,
    epochs=20,
    callbacks=[model_checkpoint_vgg, early_stopping_vgg]
)